if you want to reproduce the whole **training** procedure(not evaluating!)</br>
please note that your FOLDER PATH is correct</br>


In [ ]:

PATH = 'VRDL/VRDL_HW1_310554010' #you should key in the path of folder that contain this ipynb file
'''
for example, in my setting, I put this ipynb file folder in 'VRDL/VRDL_HW1_310554010' on My Drive
then PATH will be
'VRDL/VRDL_HW1_310554010'
'''

from google.colab import drive
drive.mount('/content/gdrive/')

!ls gdrive/MyDrive/
!nvidia-smi

import os
os.chdir('./gdrive/MyDrive/')

os.chdir(f'./{PATH}')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
 data	        DLP_final  'Meet Recordings'   tutorial.ipynb   workshop.pdf
'data mining'   hw3.ipynb   review_paper       VRDL
Tue Oct 26 02:33:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    85W / 149W |      0MiB / 11441MiB |      0%    

In [ ]:
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import birdloader
from birdloader import BirdImage as BirdImage
import function as fn
torch.manual_seed(0)

In [ ]:
BATCHSIZE = 32

trainset = BirdImage('train')
devset = BirdImage('dev')
# testset = BirdImage('test')

train_loader = DataLoader(dataset = trainset,
                          batch_size = BATCHSIZE,
                          shuffle = True,
                          num_workers = 0,
                          pin_memory = True)

dev_loader = DataLoader(dataset = devset,
                        batch_size = BATCHSIZE,
                        shuffle = False,
                        num_workers = 0,
                        pin_memory = True)

# test_loader = DataLoader(dataset = testset,
#                           batch_size = BATCHSIZE,
#                           shuffle = False,
#                           num_workers = 0,
#                           pin_memory = True)


## first model
this model is based on</br>
**model**: resnet152</br>
**loss**:cross entropy loss</br>
**LR**: 0.001</br>
**optimizer**: SGD</br>
**weight decay**: 5e-4</br>
**scheduler**: cos annealing with T_max=6</br>
**epoch**: 200 + 200(train twice continously)
</br></br>
the performance on testing data:0.62

In [ ]:
model1 = models.resnet152(pretrained=True)
model1.fc = nn.Linear(2048, 200)
info = {
    'model':model1, 
    'train_loader':train_loader, 
    'dev_loader':dev_loader, 
    'optimizer':None,
    'scheduler':None,
    'criterion':None,

    'EPOCHS':200, 
    'MODELNAME':'resnet'
}

info['optimizer'] = torch.optim.SGD(filter(lambda p: p.requires_grad, model1.parameters()), lr=0.001, weight_decay=5e-4)
info['scheduler'] = optim.lr_scheduler.CosineAnnealingLR(info['optimizer'], T_max=6, verbose=True)
fn.train(**info)


NameError: ignored

## second model
this model is based on</br>
**model**: resnet152</br>
**loss**: cos loss + cross entropy loss</br>
**LR**: 0.001</br> 
**optimizer**: SGD</br>
**weight decay**: 5e-4</br>
**scheduler**: cos annealing with T_max=6</br>
**epoch**: 200
</br></br>
the performance on testing data:0.61

In [ ]:
model = models.resnet152(pretrained=False)
model.fc = nn.Linear(2048, 200)
info = {
    'model':model, 
    'train_loader':dev_loader, 
    'dev_loader':train_loader, 
    'optimizer':None,
    'scheduler':None,
    'criterion':None,

    'EPOCHS':200, 
    'MODELNAME':'cosloss'
}
info['criterion'] = nn.CosineSimilarity()
# info['optimizer'] = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)
info['optimizer'] = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001, weight_decay=5e-4)
info['scheduler'] = optim.lr_scheduler.CosineAnnealingLR(info['optimizer'], T_max=6, verbose=True)
fn.train_new(**info)


## third model
this model finetune model2 with regarding validation set as training set
for 10 epoch,</br>and reduce the learning rate to 0.0001</br></br>
the performance on testing data:0.63
this model is based on</br>
**model**: resnet152</br>
**loss**: cos loss + cross entropy loss</br>
**LR**: 0.0001</br>
**optimizer**: SGD</br>
**weight decay**: 5e-4</br>
**scheduler**: exponential with gamma=0.99</br>
**epoch**: 10
</br></br>
the performance on testing data:0.63

In [ ]:
model = models.resnet152(pretrained=False)
model.fc = nn.Linear(2048, 200)
model.load_state_dict(torch.load('./model-para/ep8 acc1.00 0.65.pt', map_location="cuda:0"))
info = {
    'model':model, 
    'train_loader':dev_loader, 
    'dev_loader':train_loader, 
    'optimizer':None,
    'scheduler':None,
    'criterion':None,

    'EPOCHS':10, 
    'MODELNAME':'finetune1'
}
info['criterion'] = nn.CosineSimilarity()
# info['optimizer'] = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=LR)
info['optimizer'] = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, weight_decay=5e-4)
info['scheduler'] = optim.lr_scheduler.ExponentialLR(info['optimizer'], gamma=0.99, verbose=True)
fn.train_new(**info)


Adjusting learning rate of group 0 to 1.0000e-04.


100%|██████████| 10/10 [00:17<00:00,  1.78s/it]


Adjusting learning rate of group 0 to 9.9000e-05.
epoch 1%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.5824   acc: 57.00%
dev set=====>loss: 0.2152   acc: 95.70%



100%|██████████| 10/10 [00:17<00:00,  1.78s/it]


Adjusting learning rate of group 0 to 9.8010e-05.
epoch 2%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.4291   acc: 65.00%
dev set=====>loss: 0.2023   acc: 95.93%



...

Adjusting learning rate of group 0 to 9.2274e-05.
epoch 8%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.2472   acc: 87.33%
dev set=====>loss: 0.1915   acc: 95.33%



100%|██████████| 10/10 [00:17<00:00,  1.77s/it]


Adjusting learning rate of group 0 to 9.1352e-05.
epoch 9%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.2295   acc: 90.00%
dev set=====>loss: 0.1893   acc: 95.85%



100%|██████████| 10/10 [00:17<00:00,  1.76s/it]


Adjusting learning rate of group 0 to 9.0438e-05.
epoch 10%%%%%%%%%%%%%%%%%%%%
train set===>loss: 0.2139   acc: 90.67%
dev set=====>loss: 0.1914   acc: 95.11%

